In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
import torchxrayvision as xrv
from torch.nn import Identity
from torchinfo import summary
from utils.model import FusionModel
from utils.data import get_data

In [2]:
# Get .png image of the architecture of the model and save it in horizontal format
def get_model_architecture(model, input_size, filename):
    summary(model, input_size=input_size, device='cpu', col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"], col_width=16, depth=1, verbose=2, batch_dim=1)
    model_image = torchvision.models.utils.make_grid(torch.zeros(1, 3, input_size[0], input_size[1]))
    torchvision.utils.save_image(model_image, filename)

model  = FusionModel(1, 4096, "se", "concat", 18)
get_model_architecture(model, (1, 224, 224), "model_architecture.png")



TypeError: FusionModel.__init__() missing 1 required positional argument: 'train_label_shape'

In [25]:
densenet = xrv.models.DenseNet(weights="densenet121-res224-all")
resnet = xrv.models.ResNet(weights="resnet50-res512-all")

In [26]:
densenet.op_threshs = None
densenet.classifier = Identity()

In [27]:
resnet.op_threshs = None
resnet.model.fc = Identity()

In [28]:
from attention import CBAMBlock, CoordAtt, ECAAttention, ExternalAttention, SEAttention

In [4]:
randval = torch.randn(1, 1, 224, 224)

In [31]:
cbam = CBAMBlock(1024)
x = cbam(feat)
print(x.shape)

torch.Size([1, 1024, 7, 7])


In [32]:
coord = CoordAtt(1024, 1024)
x = coord(feat)
print(x.shape)

torch.Size([1, 1024, 7, 7])


In [33]:
eca = ECAAttention()
x = eca(feat)
print(x.shape)

torch.Size([1, 1024, 7, 7])


In [35]:
ea = ExternalAttention(d_model=1024, S=8)
x = ea(feat)
print(x.shape)

torch.Size([1, 1024, 7, 7])


In [36]:
sae = SEAttention(channel=1024, reduction=8)
x = sae(feat)
print(x.shape)

torch.Size([1, 1024, 7, 7])


In [39]:
multiplied_densenet = feat * x

In [40]:
multiplied_densenet.shape

torch.Size([1, 1024, 7, 7])

In [42]:
feat = resnet.features2(randval)
print(feat.shape)

torch.Size([1, 2048, 1, 1])


In [23]:
mult = torch.multiply(feat, x)
mult.shape

torch.Size([1, 2048, 1, 1])

In [43]:
sae = SEAttention(channel=2048, reduction=8)
x = sae(feat)
print(x.shape)

torch.Size([1, 2048, 1, 1])


In [44]:
multiplied_resnet = feat * x

In [46]:
resnet_tensor = F.interpolate(multiplied_resnet, size=(7, 7), mode='bilinear', align_corners=False)

In [47]:
concatenated = torch.cat([resnet_tensor, multiplied_densenet], dim=1)

In [48]:
concatenated.shape

torch.Size([1, 3072, 7, 7])

In [49]:
conv1x1 = nn.Conv2d(1024, 2048, kernel_size=1)
multiplied_densenet = conv1x1(multiplied_densenet)

In [51]:
maximum = torch.max(resnet_tensor, multiplied_densenet)
summation = resnet_tensor + multiplied_densenet

In [52]:
maximum.shape, summation.shape

(torch.Size([1, 2048, 7, 7]), torch.Size([1, 2048, 7, 7]))

In [7]:
model = Model(attention_type='se', fusion_method='max')

In [24]:
randval = torch.randn(1, 4096, 7, 7)

In [27]:
# Create a simple sequential model with few conv layers then flatten 18 output channels with softmax function

model = nn.Sequential(
    nn.Conv2d(4096, 2048, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(2048, 1024, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(1024, 18, kernel_size=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Softmax(dim=1)
)

In [4]:
dataset = get_data(
    "nih",
    "./nihcc"
)

./nihcc/images-512-NIH
['Emphysema', 'Infiltration', 'Consolidation', 'Effusion', 'Hernia', 'Edema', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax', 'Mass', 'Fibrosis', 'Atelectasis', 'Cardiomegaly', 'Nodule']


In [5]:
train_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=8,
        shuffle=True,
        num_workers=4,
        pin_memory="mps",
    )

In [4]:
model = FusionModel("se", "max", 14)

In [8]:
model.avgpool.weight

AttributeError: 'AdaptiveAvgPool2d' object has no attribute 'weight'